### Installation

In [1]:
%%capture
# Skip restarting message in Colab
#import sys; modules = list(sys.modules.keys())
#for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

In [1]:
!nvidia-smi

Fri Apr 18 07:23:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77                 Driver Version: 565.77         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:C3:00.0 Off |                  Off |
|  0%   35C    P8             18W /  230W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install unsloth #2025.3.19
!pip install vllm==0.8.2
!pip install --upgrade pillow #11.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB

In [9]:
!pip install bitsandbytes==0.45.5


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import os
os.environ["VLLM_USE_V1"] = '0'

### Loading the model

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-18 13:12:15 [__init__.py:239] Automatically detected platform cuda.
Standard import failed for UnslothDPOTrainer: No module named 'UnslothDPOTrainer'. Using tempfile instead!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [6]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
saving_name= "moslehGen/"+model_name+"-500Steps"

In [13]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)


Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/deepseek-r1-distill-qwen-14b-unsloth-bnb-4bit
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.573 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Unsloth: vLLM loading unsloth/deepseek-r1-distill-qwen-14b-bnb-4bit with actual GPU utilization = 59.38%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.15 GB. Also swap space = 6 GB.
INFO 04-18 07:32:00 [config.py:585] This model supports multiple tasks: {'classify', 'score', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection'], 'llm_int8_threshold': 6.0}
INFO 04-18 07:32:00 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-18 07:32:36 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-18 07:32:36 [model_runner.py:1146] Model loading took 9.5752 GB and 31.938059 seconds
INFO 04-18 07:32:43 [worker.py:267] Memory profiling takes 6.22 seconds
INFO 04-18 07:32:43 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.57GiB) x gpu_memory_utilization (0.59) = 14.00GiB
INFO 04-18 07:32:43 [worker.py:267] model weights take 9.58GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.88GiB; the rest of the memory reserved for KV Cache is 3.50GiB.
INFO 04-18 07:32:43 [executor_base.py:111] # cuda blocks: 1193, # CPU blocks: 2048
INFO 04-18 07:32:43 [executor_base.py:116] Maximum concurrency for 512 tokens per request: 37.28x
INFO 04-18 07:32:49 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If o

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:34<00:00,  1.52s/it]

INFO 04-18 07:33:23 [model_runner.py:1570] Graph capturing finished in 35 secs, took 0.76 GiB
INFO 04-18 07:33:23 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 47.16 seconds



Unsloth 2025.3.19 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


### Data Prep
<a name="Data"></a>



In [15]:
dataPath = "converted_data.json"

In [16]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    #data = load_dataset("openai/gsm8k", "main", split=split).select(range(1))
    data = load_dataset("json", data_files=dataPath, split="train")
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['Question']}
        ],
        'answer': extract_hash_answer(x['Response'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/25263 [00:00<?, ? examples/s]

In [17]:


dataset.remove_columns(['Complex_CoT','answer'])

Dataset({
    features: ['Question', 'Response', 'prompt'],
    num_rows: 25263
})

In [18]:
dataset["Response"][0]

'"Der Skipass ist jetzt sooo teuer."'

## Define Reward Function
We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [19]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
## Train the model

Now set up GRPO Trainer and all configurations!

In [20]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 500,
    save_steps = 100,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,263 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 137,625,600/14,000,000,000 (0.98% trained)


-------------------- Question:
Wie bewerten irische Urlauber im Alter von 60 Jahren den Zustand der Gastronomie während ihres Österreich-Urlaubs in der Vorsaison? 
Answer:
None 
Response:
Okay, so I need to answer how 60-year-old Irish vacationers evaluate the state of the gastronomy during their summer vacation in Austria. Hmm, where do I start? Well, first, I should think about the factors that affect their experience. 

Gastronomy in Austria is known for its traditional cuisine, which might include things like schnitzel, wiener schnitte, and kaiserschmarrn. Irish tourists might appreciate the quality and variety, especially since Austria is known for its regional dishes. But wait, does the Vorsaison, meaning pre-season, affect this? Pre-season could mean that some places aren't as busy, so maybe service is slower or there are fewer options. Or maybe it's the opposite, with more authentic experiences since it's not peak season.

 LANGUAGE BARRIER: Irish travelers might speak English,

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.020833,0.051031,200.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,200.000000,0.000604,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.020833,0.051031,200.000000,0.000597,0.020833,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,200.000000,0.000607,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,200.000000,0.000800,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,200.000000,0.000690,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,200.000000,0.000568,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Wie bewerten österreichische Urlauber im Alter von 71 Jahren den allgemeinen Aufenthalt in Österreich? 
Answer:
None 
Response:
Okay, so I need to figure out how Austrian vacationers aged 71 evaluate their overall stay in Austria. Hmm, I'm not exactly sure where to start, but maybe I can break it down step by step.

First, I should consider what factors are important when someone is evaluating their holiday. Probably things like the destination, the accommodation, the activities available, the food, and maybe even the weather. For a 71-year-old, safety and accessibility might be more important than for younger people.

I should think about where these seniors go. Maybe they prefer places that are not too strenuous, like cozy towns or regions with easy access to nature, such as the Alps or lake areas. They might look for places with good healthcare facilities just in case.

Accommodation is another factor. Would they prefer hotels with elevators and maybe 

<a name="Inference"></a>
## Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)

output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
## Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged(saving_name, tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged(saving_name, tokenizer, save_method = "merged_16bit", token = "YOUR_HF_TOKEN",)
        #model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "")
# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged(saving_name, tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "YOUR_HF_TOKEN")

In [ ]:
#if True: model.save_pretrained_merged(saving_name, tokenizer, save_method = "lora",)
#if True: model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "YOUR_HF_TOKEN")

## GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model-gguf", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if True: model.save_pretrained_gguf("model-gguf-16", tokenizer, quantization_method = "f16")
if True: model.push_to_hub_gguf(saving_name+"-GGUF", tokenizer, quantization_method = "f16", token = "YOUR_HF_TOKEN")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")



In [ ]:
# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        saving_name+"-multiple-GGUF", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "YOUR_HF_TOKEN",
    )

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 334.73 out of 503.53 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 48/48 [01:19<00:00,  1.66s/it] 


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at moslehGen/deepseek-ai/DeepSeek-R1-Distill-Qwen-14B-500Steps-multiple-GGUF into bf16 GGUF format.
The output location will be /workspace/moslehGen/deepseek-ai/DeepSeek-R1-Distill-Qwen-14B-500Steps-multiple-GGUF/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: DeepSeek-R1-Distill-Qwen-14B-500Steps-multiple-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 131072
INFO:hf-to

RuntimeError: *** Unsloth: Failed compiling llama.cpp with WARNING:hf-to-gguf:**************************************************************************************
. Please report this ASAP!

In [5]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

saved_model_named = "moslehGen/DeepSeek-R1-Distill-Qwen-14B-500Steps"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = saved_model_named,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.573 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Your GPU cannot handle sequence lengths of 256 due to limited GPU memory.
Unsloth: Your GPU can only handle approximately the maximum sequence length of 256.
Unsloth: vLLM loading moslehGen/DeepSeek-R1-Distill-Qwen-14B-500Steps with actual GPU utilization = 59.38%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 256. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.0 GB. Also swap space = 6 GB.
INFO 04-18 13:14

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 04-18 13:15:58 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-18 13:15:58 [model_runner.py:1146] Model loading took 9.5748 GB and 81.925209 seconds
INFO 04-18 13:16:01 [worker.py:267] Memory profiling takes 2.21 seconds
INFO 04-18 13:16:01 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.57GiB) x gpu_memory_utilization (0.59) = 14.00GiB
INFO 04-18 13:16:01 [worker.py:267] model weights take 9.57GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.70GiB; the rest of the memory reserved for KV Cache is 3.67GiB.
INFO 04-18 13:16:01 [executor_base.py:111] # cuda blocks: 1253, # CPU blocks: 2048
INFO 04-18 13:16:01 [executor_base.py:116] Maximum concurrency for 256 tokens per request: 78.31x
INFO 04-18 13:16:07 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If o

Capturing CUDA graph shapes: 100%|██████████| 19/19 [00:22<00:00,  1.20s/it]

INFO 04-18 13:16:30 [model_runner.py:1570] Graph capturing finished in 23 secs, took 0.65 GiB
INFO 04-18 13:16:30 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 31.79 seconds


In [ ]:
# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        saving_name+"-multiple-GGUF", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "YOUR_HF_TOKEN",
    )

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 334.57 out of 503.53 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 48/48 [01:16<00:00,  1.60s/it] 


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at moslehGen/deepseek-ai/DeepSeek-R1-Distill-Qwen-14B-500Steps-multiple-GGUF into bf16 GGUF format.
The output location will be /workspace/moslehGen/deepseek-ai/DeepSeek-R1-Distill-Qwen-14B-500Steps-multiple-GGUF/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: DeepSeek-R1-Distill-Qwen-14B-500Steps-multiple-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 131072
INFO:hf-to

RuntimeError: *** Unsloth: Failed compiling llama.cpp with WARNING:hf-to-gguf:**************************************************************************************
. Please report this ASAP!